In [1]:
!pip install -q -U torch transformers peft datasets bitsandbytes trl accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90

In [1]:
from huggingface_hub import login


login()

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from huggingface_hub import login

login()

print("\nProcessing Data...")

try:
    df = pd.read_csv("emotion-emotion_69k.csv")
except FileNotFoundError:
    raise FileNotFoundError("Please upload 'emotion-emotion_69k.csv' to the Colab files sidebar!")

# Fixing the "Customer/Agent" mess
df = df[df['empathetic_dialogues'].str.contains("Customer :", na=False) &
        df['empathetic_dialogues'].str.contains("Agent :", na=False)]

def get_user_text(text):
    if "Customer :" in text:
        text = text.split("Customer :")[1]
    if "Agent :" in text:
        text = text.split("Agent :")[0]
    return text.strip()

# Creating clean columns
df['clean_prompt'] = df['empathetic_dialogues'].apply(get_user_text)
df['clean_response'] = df['labels'].astype(str).str.strip()

# No lazy/short responses
initial_count = len(df)
df = df[df['clean_response'].str.len() > 15]   # Removing "Ok", "What?"
df = df[df['clean_response'].str.len() < 200]  # Removing huge paragraphs
print(f"Filtered out {initial_count - len(df)} low-quality rows.")


dataset = Dataset.from_pandas(df[['clean_prompt', 'clean_response']])

model_id = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Format for Chat Training
def format_chat(row):
    return {
        "text": tokenizer.apply_chat_template(
            [
                {"role": "system", "content": "You are a supportive, empathetic mental health assistant."},
                {"role": "user", "content": row['clean_prompt']},
                {"role": "assistant", "content": row['clean_response']}
            ],
            tokenize=False
        )
    }

# Select 5000 rows
dataset = dataset.select(range(min(5000, len(dataset)))).map(format_chat)
print("Data ready and formatted!")


Processing Data...
Filtered out 3306 low-quality rows.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Data ready and formatted!


In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

# Quantization Config (4-bit loading for free GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


print(f"Loading {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# LoRA Configuration (Fine-Tuning Adapter)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)
print("Model loaded!")

Loading Qwen/Qwen2.5-1.5B-Instruct...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded!


In [4]:
import trl
from trl import SFTTrainer, SFTConfig

# 1. Training Configuration
# Note: We use SFTConfig and REMOVE 'max_seq_length' to prevent version conflicts
sft_config = SFTConfig(
    output_dir="./empathy-bot-v1",
    dataset_text_field="text",       # Points to the column created
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=200,
    fp16=True,
    save_strategy="no",
    report_to="none",
    packing=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=sft_config,
    peft_config=peft_config,
)


print("Starting training...")
trainer.train()
print("Training complete!")

Adding EOS to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Starting training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.777500
20,2.682900
30,2.161500
40,1.959000
50,1.890000
60,1.898500
70,1.831800
80,1.802900
90,1.726700
100,1.727100


Training complete!


In [12]:
import torch

def chat_with_bot():
    print(" Mental Health Bot  - Type 'quit' to stop")
    print("-" * 50)

    # --- 1. SAFETY GUARDRAILS (The "Red Line") ---
    # We catch these keywords in Python before the AI sees them.
    # Note: Simple keyword matching can have false positives (e.g., "My plant died"),
    # but for a safety-first bot, over-blocking is better than under-blocking.
    crisis_keywords = [
        "suicide", "kill myself", "hurt myself",
        "end it all", "ending it all", "want to die",
        "kill me", "take my own life"
    ]

    crisis_response = (
        "Bot: I'm so sorry you're feeling this way. I am an AI, not a human, and I can't provide the crisis support you need. "
        "Please, reach out to a professional immediately. You are not alone, and there is help available.\n"
        "       (International Helplines: https://findahelpline.com/)"
    )

    # --- 2. SYSTEM PROMPT (The "Personality") ---
    # We instruct it to be a listener, not an advisor.
    # This prompts the model to validate feelings and ask questions rather than fixing problems.
    system_instruction = (
        "You are a compassionate, empathetic listener. "
        "Your goal is to validate the user's feelings without judging or dismissing them. "
        "Avoid making assumptions about the user's life (e.g., do not assume they have friends or family). "
        "Instead of offering solutions, ask gentle, open-ended questions to help them explore their emotions."
    )

    model.eval()

    try:
        while True:
            user_input = input("You: ")

            # Exit Conditions
            if user_input.lower() in ["quit", "exit", "bye"]:
                print("Bot: Take care. Remember to be kind to yourself. 👋")
                break

            # Empty input check
            if not user_input.strip():
                continue


            # If any crisis keyword is found, print the safe message and SKIP model generation
            if any(keyword in user_input.lower() for keyword in crisis_keywords):
                print(crisis_response)
                print("-" * 50)
                continue


            messages = [
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": user_input}
            ]

            # Tokenize and format
            input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to("cuda")

            attention_mask = torch.ones_like(input_ids).to("cuda")

            # Generate response
            with torch.no_grad():
                with torch.autocast("cuda"):
                    outputs = model.generate(
                        input_ids,
                        attention_mask=attention_mask,
                        max_new_tokens=150,     # Allow enough space for a thoughtful reply
                        temperature=0.6,        # Balance between creativity and focus
                        do_sample=True,
                        pad_token_id=tokenizer.eos_token_id
                    )

            # Decode response
            response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

            print(f"Bot: {response}")
            print("-" * 50)

    except KeyboardInterrupt:
        print("\n\nBot: Chat stopped by user. Goodbye! 👋")

# Run the chat loop
chat_with_bot()

 Mental Health Bot  - Type 'quit' to stop
--------------------------------------------------
You: I'm really terrified about my presentation tomorrow
Bot: Oh no! I'm sure it will be great!
--------------------------------------------------
You: I finally finished that painting I've been working on for months!
Bot: That's great! What did you paint?
--------------------------------------------------
You: I feel hopeless and I want to end it all.
Bot: I'm so sorry you're feeling this way. I am an AI, not a human, and I can't provide the crisis support you need. Please, reach out to a professional immediately. You are not alone, and there is help available.
       (International Helplines: https://findahelpline.com/)
--------------------------------------------------


Bot: Chat stopped by user. Goodbye! 👋
